<h4><i>Axing tedious code</i></h4>

In [1]:
data Query = Query
data SomeObj = SomeObj
data IoOnlyObj = IoOnlyObj
data Err = Err

In [7]:
decodeFn :: String -> Either Err SomeObj
decodeFn = undefined

fetchFn :: Query -> IO [String]
fetchFn = undefined

makeIoOnlyObj :: [SomeObj] -> IO [(SomeObj, IoOnlyObj)]
makeIoOnlyObj = undefined

In [8]:
pipelineQuery :: Query -> IO (Either Err [(SomeObj, IoOnlyObj)])
pipelineQuery query = do
    a <- fetchFn query
    case sequence (map decodeFn a) of
        (Left err) -> return $ Left err
        (Right res) -> do
            a <- makeIoOnlyObj res
            return $ Right a

In [9]:
pipelineQuery' :: Query -> IO (Either Err [(SomeObj, IoOnlyObj)])
pipelineQuery' = ((traverse makeIoOnlyObj . traverse decodeFn) =<<) . fetchFn